In [1]:
#imports
import numpy as np
import math

In [3]:
# Load Data
data = np.genfromtxt('scale-d-10d.csv', delimiter=' ')
data = data[:,0:9]

labels = np.genfromtxt('scale-d-10d.csv', delimiter=' ',dtype="|U5")
labels = labels[:,10]

In [8]:
#dependencies: numpy
class P3C:

    #Poisson threshold is the only parameter for P3C 
    def __init__(self, poisson_threshold): 
        self.poisson_threshold = poisson_threshold
        self.alpha = 0.001
        
        #sklearn-like attributes
        self.labels_ = None
        self.cluster_centers_ = None #"cluster cores"
        
    #data X is numpy.ndarray with samples x features (no label!)  
    def fit (X):
        #all non-object variables only used in fit (X) go here...
        marked_bins = [] #marked bins: nested lists: 1st level: attributes, 2nd level:
                         #different intervals, 3rd level: start and end of inteval.
                         #used as interface between part 3.1 and 3.2
        
        
        #all the method calls of 3.1 and 3.2 we have to implement go here...
        
        #self.cluster_center_ = ... #used as interface between part 3.2. and 3.3
        
    #data X is numpy.ndarray with samples x features (no label!)  
    def predict (X):
        pass #remove pass when implementing predict (X)
        
        #all the method calls of 3.3, 3.4 and 3.5 we have to implement go here...
        
        #self.labels_ = #final result of the algorithm.
        
    #data X is numpy.ndarray with samples x features (no label!)  
    def fit_predict (X):
        fit (X)
        predict (X)

In [269]:
# 3.1
def proj_true_p_signature(M):
    
    # For every bin in every attribute, its support is computed
    supp_sets = compute_support(M)
    


    return None

In [271]:
# Compute Support set of all attributes
# SupportSet = {x ∈ D | x.aj ∈ S }
def compute_support(M):
    n = M.shape[0] # n = number of data objects
    attribute_number = M.shape[1] # number of attributes 
    bin_number = int(1 + math.log(n,2)) # number of bins
    list_of_supports = [] # list of all supports
    
    for i in range(attribute_number):
        supp_set = [[]for i in range(3)]
    
        aj_max = np.max(M[:,i])
        aj_min = np.min(M[:,i])
    
        interval_length = (aj_max - aj_min ) / bin_number
    
        bins = np.zeros([bin_number])
    
        for k in range(bin_number):
            bins[k] = aj_min + (k+1)*interval_length
        
        for j in range(n):
            supp_set_index = 0
            for k in range(len(bins)):
                if M[j,i] > bins[k]:
                    supp_set_index += 1
            supp_set[supp_set_index].append(M[j])
    
        list_of_supports.append(supp_set)
    
    
    return list_of_supports

In [272]:
print(data)
print(data.shape)

print(labels)
print(labels.shape)


print(int(1 + math.log(data.shape[0],2)))

[[ 0.65264191 -0.00371625  0.01521223 ...  0.21708319  0.78426444
   0.27407853]
 [ 1.01930291  0.35519293  0.47796839 ...  0.60008908  0.35612478
   0.24724175]
 [ 0.5940366  -0.33190342 -0.10154468 ...  0.68686449  0.53546747
   1.27822433]
 ...
 [ 0.84569877  0.35925799  0.64490759 ...  0.87088832  0.63797899
   0.87146842]
 [ 0.27499317  0.49355767  0.20943054 ...  0.93080848  0.72043327
   1.42526599]
 [ 0.39640662  0.57944655  0.980204   ...  0.0876678   0.15534795
   0.67756318]]
(10000, 9)
['C1' 'C1' 'C1' ... 'C100' 'C100' 'C100']
(10000,)
14


In [273]:
M = np.array([[1, 2], [4, 5], [3, 3], [6, 7]])
print(compute_support(M))

[[[array([1, 2])], [array([4, 5]), array([3, 3])], [array([6, 7])]], [[array([1, 2]), array([3, 3])], [array([4, 5])], [array([6, 7])]]]


In [274]:
# Extra
M = np.array([[1, 2], [4, 5], [3, 3], [6, 7]])
n = M.shape[0] # n = number of data objects
attribute_number = M.shape[1] # number of attributes 
bin_number = int(1 + math.log(n,2)) # number of bins
list_of_supports = [] # list of all supports

for i in range(attribute_number):
    supp_set = [[]for i in range(3)]
    
    aj_max = np.max(M[:,i])
    aj_min = np.min(M[:,i])
    interval_length = (aj_max - aj_min ) / bin_number
    bins = np.zeros([bin_number])
    for k in range(bin_number):
        bins[k] = aj_min + (k+1)*interval_length
        
    for j in range(n):
        supp_set_index = 0
        for k in range(len(bins)):
            if M[j,i] > bins[k]:
                supp_set_index += 1
        supp_set[supp_set_index].append(M[j])
    
    print(supp_set)
    


[[array([1, 2])], [array([4, 5]), array([3, 3])], [array([6, 7])]]
[[array([1, 2]), array([3, 3])], [array([4, 5])], [array([6, 7])]]


In [263]:
M = np.array([[1, 2], [4, 5], [3, 3], [6, 7]])

supp_set = [[]for i in range(3)]
supp_set[0].append(M[1])
supp_set[2].append(M[3])

supp_set

[[array([4, 5])], [], [array([6, 7])]]

[1, 2, 3, 4]

In [206]:
1 + 2*interval_length

4.333333333333334